In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/result2/output_song_with_structure.json
/kaggle/input/result2/output_song_gptno.json
/kaggle/input/result2/output_song_n_gram.json
/kaggle/input/result/output_song_baseline.json
/kaggle/input/result/test_dataset.txt


# **BLEU**

In [4]:
from nltk.tokenize import word_tokenize
import json

In [5]:
with open('/kaggle/input/result2/output_song_with_structure.json','r') as f:
    model_with_structure = json.load(f)
with open('/kaggle/input/result2/output_song_gptno.json','r') as f:
    model_no_str = json.load(f)
with open('/kaggle/input/result2/output_song_n_gram.json','r') as f:
    model_ng = json.load(f)

In [9]:
def nltkTokenizer(sentence):
    tokens = word_tokenize(sentence)
    return tokens

In [23]:
pip install --user fast-bleu

  Created wheel for fast-bleu: filename=fast_bleu-0.0.89-cp37-cp37m-linux_x86_64.whl size=641534 sha256=7fe2e0db66818132d9a79a6798cb9f213fc184a70522f66b266787c50a41ac5f
  Stored in directory: /root/.cache/pip/wheels/59/f6/b7/3b841c3f010bc5993afbc1b7d8ed78fedca89747dd735488ae
Successfully built fast-bleu
Note: you may need to restart the kernel to use updated packages.


In [ ]:
nltkTokenizer(sentence)

In [12]:
import re
with_str = []
for i in  model_with_structure['output_song_with_structure'][:-1]:
    with_str.append(nltkTokenizer(re.sub('<.{3,5}>|intro, verse, chorus','',i)))
no_str = []
for i in  model_with_structure['output_song_with_structure'][:-1]:
    no_str.append(nltkTokenizer(re.sub('\[\]','',i)))
ng = []
for i in  model_ng['output_song_n_gram']:
    ng.append(nltkTokenizer(re.sub('  |. .',' ',i)))
    

In [14]:
with open('../input/result/test_dataset.txt','r') as f:
    test_data = f.read()

In [15]:
test = []
for i in test_data.split('<Newsong>'):
    test.append(nltkTokenizer(re.sub('<.{3,5}>|<Newline>|intro,| verse,| chorus','',i)))

In [16]:
len(with_str),len(no_str),len(ng),len(test)

(358, 358, 358, 358)

In [ ]:
test

In [ ]:
i =0 

list_bleu(test[0],[no_str,ng, with_str])

In [17]:
from nltk.translate.bleu_score import corpus_bleu

In [20]:
score = 0
for i in range(0,358):
    references =  [test[i]]
    candidates =  [with_str[i]]
    score += corpus_bleu(references, candidates)
print('with structure', score/358) 

/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


with structure 0.27850996927578353


In [22]:
score = 0
for i in range(0,358):
    references =  [test[i]]
    candidates =  [no_str[i]]
    score += corpus_bleu(references, candidates)
print('with structure', score/358) 

with structure 0.3094454666305401


In [21]:
score = 0
for i in range(0,358):
    references =  [test[i]]
    candidates =  [ng[i]]
    score += corpus_bleu(references, candidates)
print('with structure', score/358) 

/opt/conda/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


with structure 0.19783657535750013


In [24]:
#this is BLEU-4

# **Jaccard**

Jointly Measuring Diversity and Quality in Text Generation Models
 https://github.com/IAmS4n/TextGenerationEvaluationMetrics/tree/789aa6141784293984ead53aea3eb579952d3f46

In [41]:
from nltk.translate.bleu_score import ngrams
from collections import Counter
from functools import reduce

import numpy as np

class metric_names:
    jaccard = "Jaccard"
    sorensen = "Sorensen"
    canberra = "Canberra"
    minkowski = "Minkowski"
    BLEU = "BLEU"
    SelfBLEU = "SelfBLEU"


def get_ngrams(sentences, n):
    f = lambda x: (list(ngrams(x, n)) if len(x) >= n else [])
    return list(map(f, sentences))


class MultisetDistances:

    def __init__(self, references, min_n=3, max_n=5):
        super().__init__()
        # print('multiset distances init upto {}!'.format(max_n))
        self.references = references
        self.max_n = max_n
        self.min_n = min_n
        assert self.max_n >= self.min_n
        assert self.min_n >= 1
        self.ref_ngrams = self._get_ngrams(references)

    def get_cached_fields(self):
        return self.ref_ngrams,

    def _get_ngrams(self, sentences):
        samples_size = len(sentences)
        all_counters = [Counter([x for y in get_ngrams(sentences, n + 1) for x in y])
                        for n in range(self.max_n)]
        for n_counter in all_counters:
            for k in n_counter.keys():
                n_counter[k] /= samples_size
        return all_counters

    def get_ngram_stuff(self, sentences):
        sample_ngrams = self._get_ngrams(sentences)
        ngrams_intersection = [sample_ngrams[i] & self.ref_ngrams[i]
                               for i in range(self.max_n)]  # intersection:  min(c[x], d[x])
        ngrams_union = [sample_ngrams[i] | self.ref_ngrams[i]
                        for i in range(self.max_n)]  # union:  max(c[x], d[x])
        ngrams_abs_diff = [ngrams_union[i] - ngrams_intersection[i] \
                           for i in range(self.max_n)]
        ngrams_added = [sample_ngrams[i] + self.ref_ngrams[i]
                        for i in range(self.max_n)]

        return ngrams_intersection, ngrams_union, ngrams_abs_diff, ngrams_added

    def _final_average(self, score_value):
        return np.power(reduce(lambda x, y: x * y, score_value), 1. / float(len(score_value)))

    def _jaccard(self, ngrams_intersection, ngrams_union):
        jaccard_value = [float(sum(ngrams_intersection[n].values())) / sum(ngrams_union[n].values()) for n in
                         range(self.max_n)]
        return jaccard_value

    def get_jaccard_score(self, sentences):
        #print('Jaccard distances preprocess upto {}!'.format(self.max_n))
        ngrams_intersection, ngrams_union, ngrams_abs_diff, ngrams_added = self.get_ngram_stuff(sentences)

        jaccard_value = self._jaccard(ngrams_intersection=ngrams_intersection, ngrams_union=ngrams_union)

        return {n: self._final_average(jaccard_value[:n]) for n in range(self.min_n, self.max_n + 1)}

    def _sorensen(self, ngrams_abs_diff, ngrams_added):
        sorensen_value = [float(sum(ngrams_abs_diff[n].values())) / sum(ngrams_added[n].values()) for n in
                          range(self.max_n)]
        return sorensen_value

    def get_sorensen_score(self, sentences):
        print('Sorensen distances preprocess upto {}!'.format(self.max_n))
        ngrams_intersection, ngrams_union, ngrams_abs_diff, ngrams_added = self.get_ngram_stuff(sentences)

        sorensen_value = self._sorensen(ngrams_abs_diff=ngrams_abs_diff, ngrams_added=ngrams_added)

        return {n: self._final_average(sorensen_value[:n]) for n in range(self.min_n, self.max_n + 1)}

    def _canberra(self, ngrams_abs_diff, ngrams_added):
        canberra_value = [np.sum([ngrams_abs_diff[n][key] / float(ngrams_added[n][key]) for key in ngrams_abs_diff[n]])
                          for n in range(self.max_n)]
        return canberra_value

    def get_canberra_score(self, sentences):
        print('Canberra distances preprocess upto {}!'.format(self.max_n))
        ngrams_intersection, ngrams_union, ngrams_abs_diff, ngrams_added = self.get_ngram_stuff(sentences)
        canberra_value = self._canberra(ngrams_abs_diff=ngrams_abs_diff, ngrams_added=ngrams_added)
        return {n: self._final_average(canberra_value[:n]) for n in range(self.min_n, self.max_n + 1)}

    def _minkowski(self, ngrams_abs_diff, p):
        minkowski_value = [np.power(np.sum(np.power(list(ngrams_abs_diff[n].values()), p)), 1. / p) for n in
                           range(self.max_n)]
        return minkowski_value

    def get_minkowski_score(self, sentences, p):
        print('Minkowski (p={}) distances preprocess upto {}!'.format(p, self.max_n))
        ngrams_intersection, ngrams_union, ngrams_abs_diff, ngrams_added = self.get_ngram_stuff(sentences)

        minkowski_value = self._minkowski(ngrams_abs_diff=ngrams_abs_diff, p=p)

        return {n: self._final_average(minkowski_value[:n]) for n in range(self.min_n, self.max_n + 1)}

    def get_all_score(self, sentences, max_mikowski_order=3):
        print('multiset distances preprocess upto {}!'.format(self.max_n))
        ngrams_intersection, ngrams_union, ngrams_abs_diff, ngrams_added = self.get_ngram_stuff(sentences)

        temp_results = {}

        print('multiset distances evaluating upto {}!'.format(self.max_n))
        temp_results[metric_names.jaccard] = self._jaccard(ngrams_intersection=ngrams_intersection,
                                                           ngrams_union=ngrams_union)
        temp_results[metric_names.sorensen] = self._sorensen(ngrams_abs_diff=ngrams_abs_diff, ngrams_added=ngrams_added)
        temp_results[metric_names.canberra] = self._canberra(ngrams_abs_diff=ngrams_abs_diff, ngrams_added=ngrams_added)
        for p in range(1, max_mikowski_order + 1):
            temp_results['p%d-%s' % (p, metric_names.minkowski)] = self._minkowski(ngrams_abs_diff=ngrams_abs_diff, p=p)

        result = {}
        for key in temp_results:
            for n in range(self.min_n, self.max_n + 1):
                result[key + '%d' % n] = self._final_average(temp_results[key][:n])
        return result

In [42]:
msj_distance

{3: 0.0, 4: 0.0, 5: 0.0}

In [45]:
score = 0

for i in range(0,358,5):
    references =  test[i:i+5]
    candidates =  with_str[i:i+5]

    msd = MultisetDistances(references=references)
    msj_distance = msd.get_jaccard_score(sentences=candidates)
    score += msj_distance[3]
    #fbd = FBD(references=references, model_name="bert-base-uncased", bert_model_dir="/tmp/Bert/")

print('with structure')
print('msj_distance',score/len(range(0,358,5)))
score = 0
for i in range(0,358,5):
    references =  test[i:i+5]
    candidates =  no_str[i:i+5]

    msd = MultisetDistances(references=references)
    msj_distance = msd.get_jaccard_score(sentences=candidates)
    score += msj_distance[3]
print('no structure','msj_distance',score/len(range(0,358,5)))
score = 0
for i in range(0,358,5):
    references =  test[i:i+5]
    candidates =  ng[i:i+5]

    msd = MultisetDistances(references=references)
    msj_distance = msd.get_jaccard_score(sentences=candidates)
    score += msj_distance[3]
print('n-gram','msj_distance',score/len(range(0,358,5)))

with structure
msj_distance 0.014830080896122248
no structure msj_distance 0.01470584000742136
n-gram msj_distance 0.0
